<a href="https://colab.research.google.com/github/dianjin0407/BA820-Group4-Job-Listing-Integrity-Investigation/blob/main/BA820_B1_Group4_Deliverable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Listing Integrity Investigation

**Group 4:** Dian Jin, Mingze Wu, Tanvi Sheth, Sneha Sunil Ekka, Jenil Shah

## Importing Libraries

# Part 1
---

In [18]:
# Data Structures
import numpy  as np
import pandas as pd

# Corpus Processing
import re
# import string
import nltk
import nltk.corpus
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# Visualization and Analysis
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.metrics import silhouette_samples, silhouette_score

# K-Means
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

colors = px.colors.qualitative.Pastel
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading the Data

In [19]:
!pip install opendatasets

In [20]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction?resource=download")

Skipping, found downloaded files in "./real-or-fake-fake-jobposting-prediction" (use force=True to force download)


In [21]:
!pip install your-package-name

In [22]:
data = pd.read_csv('/content/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')
data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

## Cleaning the Data

In [24]:
# Checking for NULLs in the data
nan_counts = data.isnull().sum()
nan_percent = data.isnull().sum()/data.shape[0]
nans_dict = {'count_of_nans':nan_counts, 'percent_of_nans':nan_percent}

pd.DataFrame(nans_dict).sort_values('percent_of_nans')

,count_of_nans,percent_of_nans
job_id,0,0.000000
has_questions,0,0.000000
has_company_logo,0,0.000000
telecommuting,0,0.000000
fraudulent,0,0.000000
title,0,0.000000
description,1,0.000056
location,346,0.019351
requirements,2695,0.150727
company_profile,3308,0.185011


In [25]:
# Dropping NAN value in the 'description' column
data.dropna(subset=['description'], inplace=True)
data.shape

(17879, 18)

In [26]:
# Verifying the categorical nature of some columns
columns = ['telecommuting','has_company_logo','has_questions','employment_type','required_experience','required_education','fraudulent']
cat_check = data[columns]
unique = cat_check.apply(lambda col: col.unique())
pd.DataFrame(unique, columns=['Unique Values'])

,Unique Values
telecommuting,"[0, 1]"
has_company_logo,"[1, 0]"
has_questions,"[0, 1]"
employment_type,"[Other, Full-time, nan, Part-time, Contract, T..."
required_experience,"[Internship, Not Applicable, nan, Mid-Senior l..."
required_education,"[nan, Bachelor's Degree, Master's Degree, High..."
fraudulent,"[0, 1]"


In [27]:
# Filling missing values in categorical columns with "Not Specified"
categorical_columns = ['employment_type', 'required_experience', 'required_education']
for column in categorical_columns:
    data[column].fillna('Not Specified', inplace=True)

# Collapsing 'Unspecified' and 'Not Specified' in the 'required_education' column into a single category
data.loc[data['required_education'] == 'Unspecified', 'required_education'] = 'Not Specified'

In [28]:
# Changing the datatypes of categorical columns to 'category'
data = data.astype({'telecommuting':'category', 'has_company_logo':'category', 'has_questions':'category', 'fraudulent':'category',
                    'employment_type':'category', 'required_experience':'category', 'required_education':'category'})

In [29]:
# Filling missing textual columns with empty strings
textual_columns = ['company_profile', 'description', 'requirements', 'benefits']
for column in textual_columns:
    data[column].fillna('', inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17879 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   job_id               17879 non-null  int64   
 1   title                17879 non-null  object  
 2   location             17533 non-null  object  
 3   department           6333 non-null   object  
 4   salary_range         2868 non-null   object  
 5   company_profile      17879 non-null  object  
 6   description          17879 non-null  object  
 7   requirements         17879 non-null  object  
 8   benefits             17879 non-null  object  
 9   telecommuting        17879 non-null  category
 10  has_company_logo     17879 non-null  category
 11  has_questions        17879 non-null  category
 12  employment_type      17879 non-null  category
 13  required_experience  17879 non-null  category
 14  required_education   17879 non-null  category
 15  industry           

In [30]:
# Checking for NULLs again
nan_counts = data.isnull().sum()
nan_percent = data.isnull().sum()/data.shape[0]
nans_dict = {'count_of_nans':nan_counts, 'percent_of_nans':nan_percent}

nans_df = pd.DataFrame(nans_dict).sort_values('percent_of_nans')
nans_df[nans_df.count_of_nans > 0]

,count_of_nans,percent_of_nans
location,346,0.019352
industry,4902,0.274176
function,6454,0.360982
department,11546,0.645786
salary_range,15011,0.839588


In [31]:
## Investigating the 'salary_range' column
data['salary_range'].unique()

array([nan, '20000-28000', '100000-120000', '120000-150000',
       '50000-65000', '40000-50000', '60-80', '65000-70000', '75-115',
       '75000-110000', '17000-20000', '16000-28000', '95000-115000',
       '15000-18000', '50000-70000', '45000-60000', '30000-40000',
       '70000-90000', '10000-14000', '50-110', '28000-45000', '0-34300',
       '35000-40000', '9-Dec', '44000-57000', '18500-28000',
       '55000-75000', '30000-35000', '0-0', '20000-40000',
       '360000-600000', '50000-80000', '80000-100000', '52000-78000',
       '15750-15750', '40000-65000', '45000-50000', '30000-37000',
       '45000-67000', '35000-100000', '180000-216000', '45000-65000',
       '28000-32000', '0-1000', '36000-40000', '80000-110000',
       '35000-73000', '19000-19000', '60000-120000', '120000-15000000',
       '42000-55000', '90000-120000', '100000-150000', '28000-38000',
       '1600-1700', '50000-60000', '30000-70000', '32000-40000', '50-100',
       '9000-17000', '23040-28800', '105-110', '1300

In [32]:
# Checking no of rows that have bad values in 'salary_range'
salary_not_null = data[~ data['salary_range'].isna()]
salary_check_df = salary_not_null[salary_not_null['salary_range'].str.contains('[a-zA-Z]')]
salary_check_df.shape

(26, 18)

In [33]:
# Getting rid of the above rows that contain dates in the 'salaray_range'
indices_to_drop = salary_check_df.index

# Obtaining the cleaned data
data.drop(indices_to_drop, inplace=True)

# Resetting the index
data.reset_index(drop=True, inplace=True)
data.shape

(17853, 18)

In [34]:
## Investigating the 'department' column
list(data['department'].unique())

['Marketing',
 'Success',
 nan,
 'Sales',
 'ANDROIDPIT',
 'HR',
 ' R&D',
 'Engagement',
 'Businessfriend.com',
 'Medical',
 'Field',
 'All',
 'Design',
 'Production',
 'ICM',
 'General Services',
 'Engineering',
 'IT',
 'Business Development',
 'Human Resources',
 'Oil & Energy',
 'Marketplace',
 'Cloud Services',
 'FP',
 'Client Services',
 'Operations',
 'Materials',
 'tech',
 'Sales and Business Development',
 'R&D',
 'Development',
 'Incubation Services',
 'Field Operations',
 'MKT',
 'Technology',
 'Power Plant & Energy',
 'Approvals Department',
 'Playfair Capital',
 'Development ',
 'Tech',
 'Software development',
 'Media',
 'Line-Up',
 'Management',
 'Squiz ',
 'Finance',
 'Financial',
 'Retail',
 'Marketing and Communications',
 'Research',
 'Connectivity',
 'PMO',
 'Product',
 'Student Beans Mag',
 'Information Technology Group',
 'DTVMA',
 'G&A',
 'Implementations',
 'OPS',
 'Partnership Management',
 'Professional Services',
 'Customer Care',
 'Account Management',
 'EC',


In [35]:
# Dropping the 'department' column
data.drop('department', axis=1, inplace=True)

In [36]:
## Investigating the 'function' column
data['function'].unique()

array(['Marketing', 'Customer Service', nan, 'Sales',
       'Health Care Provider', 'Management', 'Information Technology',
       'Other', 'Engineering', 'Administrative', 'Design', 'Production',
       'Education', 'Supply Chain', 'Business Development',
       'Product Management', 'Financial Analyst', 'Consulting',
       'Human Resources', 'Project Management', 'Manufacturing',
       'Public Relations', 'Strategy/Planning', 'Advertising', 'Finance',
       'General Business', 'Research', 'Accounting/Auditing',
       'Art/Creative', 'Quality Assurance', 'Data Analyst',
       'Business Analyst', 'Writing/Editing', 'Distribution', 'Science',
       'Training', 'Purchasing', 'Legal'], dtype=object)

In [37]:
## Investigating the 'industry' column
data['industry'].unique()

array([nan, 'Marketing and Advertising', 'Computer Software',
       'Hospital & Health Care', 'Online Media',
       'Information Technology and Services', 'Financial Services',
       'Management Consulting', 'Events Services', 'Internet',
       'Facilities Services', 'Consumer Electronics',
       'Telecommunications', 'Consumer Services', 'Construction',
       'Oil & Energy', 'Education Management', 'Building Materials',
       'Banking', 'Food & Beverages', 'Food Production',
       'Health, Wellness and Fitness', 'Insurance', 'E-Learning',
       'Cosmetics', 'Staffing and Recruiting',
       'Venture Capital & Private Equity', 'Leisure, Travel & Tourism',
       'Human Resources', 'Pharmaceuticals', 'Farming', 'Legal Services',
       'Luxury Goods & Jewelry', 'Machinery', 'Real Estate',
       'Mechanical or Industrial Engineering',
       'Public Relations and Communications', 'Consumer Goods',
       'Medical Practice', 'Electrical/Electronic Manufacturing',
       'Hospita

In [38]:
## Investigating the 'location' column
data['location'].unique()

array(['US, NY, New York', 'NZ, , Auckland', 'US, IA, Wever', ...,
       'US, CA, los Angeles', 'CA, , Ottawa', 'GB, WSX, Chichester'],
      dtype=object)

In [39]:
# Extracting 'country' from 'location' column
data['country'] = data['location'].str[:2]

In [40]:
# Filling missing values in other columns with "Not Specified"
other_columns = ['location', 'country', 'industry', 'function', 'salary_range']
for column in other_columns:
    data[column].fillna("Not Specified", inplace=True)

In [41]:
data.isna().sum()

job_id                 0
title                  0
location               0
salary_range           0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
country                0
dtype: int64

## Preprocessing Text Data With NLP

### Functions

### Tokenization

### Vectorizarion

## EDA & Visualisations

### Exploratory Plots

In [42]:
# Plot of Distribution of Employment Types

employment_counts = data['employment_type'].value_counts().reset_index()
employment_counts.columns = ['employment_type', 'count']

# Plot using Plotly
fig = px.bar(employment_counts,
             y='employment_type',
             x='count',
             color='employment_type',
             orientation='h',
             title='Plot 1: Distribution of Employment Types',
             labels={'count': 'Count', 'employment_type': 'Employment Type'},
             color_discrete_sequence=colors[2:]
            )

fig.show()

In [43]:
# Plot of Distribution of Required Education for Job Postings

required_education = data['required_education'].value_counts().reset_index()
required_education.columns = ['required_education', 'count']

fig = px.bar(required_education,
             y='required_education',
             x='count',
             color='required_education',
             orientation='h',
             title='Plot 2: Distribution of Required Education for Job Postings',
             labels={'count': 'Count', 'required_education': 'Required Education'},
             color_discrete_sequence=colors[2:]
            )

fig.show()

In [44]:
# Plot of Top 10 Industries Represented in Job Postings

top_industries = data['industry'].value_counts().reset_index().head(10)
top_industries.columns = ['industry', 'count']

fig = px.bar(top_industries,
             y='industry',
             x='count',
             color='industry',
             orientation='h',
             title='Plot 3: Top 10 Industries Represented in Job Postings',
             labels={'count': 'Count', 'industry': 'Industry'},
             color_discrete_sequence=colors[2:]
            )
fig.show()

In [45]:
# Plotting Distribution of Job Postings With and Without Company Logo

# Creating a copy of data for plots
plot_df = data.copy()
plot_df['fraudulent'] = plot_df['fraudulent'].map({0: "Real", 1: "Fake"})

fig = px.histogram(plot_df, x='has_company_logo', color='fraudulent', barmode='group', category_orders={'has_company_logo': ['f', 't']}, color_discrete_sequence=colors)

fig.update_layout(
    title='Plot 4: Distribution of Job Postings With and Without Company Logo',
    xaxis_title='Has Company Logo',
    yaxis_title='Count',
    width=800,
    height=600,
    legend_title_text='Fraudulent'
)
fig.update_layout(xaxis=dict(tickvals=[0, 1], ticktext=['No', 'Yes']))

fig.show()

In [46]:
# Plotting Distribution of Telecommuting Job Postings
fig = px.histogram(plot_df, x='telecommuting', color='fraudulent', barmode='group', category_orders={'telecommuting': ['f', 't']}, color_discrete_sequence=colors)

fig.update_layout(
    title='Plot 5: Distribution of Telecommuting Job Postings',
    xaxis_title='Telecommuting',
    yaxis_title='Count',
    width=800,
    height=600,
    legend_title_text='Fraudulent'
)
fig.update_layout(xaxis=dict(tickvals=[0, 1], ticktext=['No', 'Yes']))

fig.show()

In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df1 = data.copy()

# removing not specified from the country column
df1 = df1[df1['country'] != 'Not Specified']

country_counts = df1.groupby(['country', 'fraudulent']).size().unstack(fill_value=0)

country_counts['total'] = country_counts.sum(axis=1)

country_counts = country_counts.sort_values(by='total', ascending=False)

top_fraudulent_countries, top_non_fraudulent_countries = [country_counts.nlargest(10, 1), country_counts.nlargest(10, 0)]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Top 10 Fraudulent Countries", "Top 10 Non-Fraudulent Countries"))


fig.add_trace(go.Bar(x=top_fraudulent_countries.index, y=top_fraudulent_countries[1], name='Fraudulent', marker = {'color' : colors[0]}),
              row=1, col=1)

fig.add_trace(go.Bar(x=top_non_fraudulent_countries.index, y=top_non_fraudulent_countries[0], name='Non-Fraudulent', marker = {'color' : colors[1]}),
              row=1, col=2)

fig.update_layout(title="Plot 6: Top 10 Fraudulent and Non-Fraudulent Countries", showlegend=False)
fig.show()

### Word Clouds for Text Columns

# Part 2
---

## Dimensionality Reduction

## Clustering

# Part 3
---

## Classification